In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn

from torch.utils.data import Dataset
# from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel

import timm
import tqdm

from accelerate import Accelerator, notebook_launcher # main interface, distributed launcher
from accelerate.utils import set_seed

In [2]:
torch.cuda.is_initialized()

False

In [3]:
device = 'cuda'

# Load Data

In [4]:
batch_size = 32
hidden_size = 768
img_size = 224

data_dir = '/kaggle/input/skin-disease-train/binary'

In [5]:
for folder in os.listdir(data_dir):
    print(folder, len(os.listdir(os.path.join(data_dir, folder))))

무증상 189376
유증상 186746


In [6]:
from torchvision.transforms import (
    Compose,
    ToTensor,
    RandomHorizontalFlip,
    RandomVerticalFlip
)

transforms = Compose(
        [
            ToTensor(),
            RandomHorizontalFlip(),
            RandomVerticalFlip()
        ]
    )

In [ ]:
dataset = datasets.ImageFolder(data_dir, transforms)
# train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms)
# valid_dataset = datasets.ImageFolder(os.path.join(data_dir, 'valid'), transforms)
# test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms)

In [ ]:
valid_size = int(len(dataset)*0.2)
train_size = len(dataset) - valid_size

train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

In [ ]:
print(len(train_dataset), len(valid_dataset))

# Model Training

inception_v4 </br> 
resnet50 </br>
resnetv2_50 </br>
efficientnet_b0 </br>
vit_base_resnet50_224_in21k </br>
vit_base_patch16_224 </br>
vit_base_patch8_224

In [ ]:
model_name = 'vit_base_patch8_224'
pretrained = True
model = timm.create_model(model_name, pretrained=pretrained, num_classes=hidden_size)

In [ ]:
torch.cuda.is_initialized()

In [ ]:
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
from IPython.display import clear_output

def loss_epoch_curve(train_loss_epoch, val_loss_epoch, train_acc_epoch, val_acc_epoch):

  figure, ax = plt.subplots(1, 2, figsize=(12, 5))

  ax[0].plot(train_loss_epoch)
  ax[0].plot(val_loss_epoch)
  ax[0].set_title('Loss-Epoch curve')
  ax[0].set_ylabel('Loss')
  ax[0].set_xlabel('Epoch')
  ax[0].legend(['train', 'val'], loc='upper right')

  ax[1].plot(train_acc_epoch)
  ax[1].plot(val_acc_epoch)
  ax[1].set_title('Model Accuracy')
  ax[1].set_ylabel('Accuracy')
  ax[1].set_xlabel('Epoch')
  ax[1].legend(['train', 'val'], loc='lower right')

  plt.show()

In [ ]:
class Classifier(nn.Module):
    def __init__(self, model, hidden_size, num_classes=2): 
        super().__init__()
        self.model = model
        self.classifier = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return self.classifier(x)

In [ ]:
from collections import OrderedDict
import pickle

def save_checkpoint(epoch, labels, model, optimizer, 
                    train_loss_epoch, val_loss_epoch, train_acc_epoch, val_acc_epoch,
                    model_path, filename):
    
    # model_dict = OrderedDict([(k, v) for k, v in model.state_dict().items()])
    state = {
        'epoch': epoch,
        'state_dict': model.module.state_dict(), # model_dict,
        'optimizer': optimizer.state_dict(),
        'label': labels
    }
    torch.save(state, os.path.join(model_path, f'{filename}.pt'))

    config = {"train":{"acc":train_acc_epoch, "loss":train_loss_epoch},
          "valid":{"acc":val_acc_epoch, "loss":val_loss_epoch}}

    with open(os.path.join(model_path, f'{filename}.pickle'),'wb') as fw:
        pickle.dump(config, fw)

In [ ]:
model_path = '/kaggle/working/multi'
os.makedirs(model_path, exist_ok=True)

In [ ]:
def load_dict(model, optimizer, dict_file):
    
    pretrained = torch.load(dict_file)
    
    epoch = pretrained['epoch']
    state_dict = pretrained['state_dict']
    opt_dict = pretrained['optimizer']
    labels = pretrained['label']
    
    model_dict = model.state_dict()
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    
    optimizer.load_state_dict(opt_dict)
    
    return epoch, labels, model, optimizer

In [ ]:
import pickle
def load_records(pkl_file):
    with open(pkl_file, 'rb') as f:
        records = pickle.load(f)
    return records['train'], records['valid']

In [ ]:
file_path = '/kaggle/input/dog-skin-multimodel/multi'

pretrained = False

In [ ]:
labels = train_dataset.classes
labels

In [ ]:
def main_worker(model, hidden_size, labels, model_path, model_name, num_epochs, mixed_precision='fp16', batch_size=batch_size):
    
    num_epochs = num_epochs
    # scaler = torch.cuda.amp.GradScaler()

    best_val_acc, best_val_loss = 0.0, 100.0

    train_loss_epoch, val_loss_epoch = [], []
    train_acc_epoch, val_acc_epoch = [], []
    epoch_start = 0

    if pretrained:
        dict_file = os.path.join(file_path, f'{model_name}.pt')
        pkl_file = os.path.join(file_path, f'{model_name}.pickle')

        epoch_start, labels, model, optimizer = load_dict(model, optimizer, dict_file)
        train_epoch, valid_epoch = load_records(pkl_file)
        train_loss_epoch, train_acc_epoch = train_epoch['loss'], train_epoch['acc']
        val_loss_epoch, val_acc_epoch = valid_epoch['loss'], valid_epoch['acc']

    accelerator = Accelerator(mixed_precision=mixed_precision, log_with='wandb')
    
    # train_sampler = DistributedSampler(train_dataset)
    # valid_sampler = DistributedSampler(valid_dataset)
    
    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=4)
                              # pin_memory=True,
                              # sampler=train_sampler)

    valid_loader = DataLoader(valid_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=4)
                              # pin_memory=True,
                              # sampler=valid_sampler)
    
    classifier = Classifier(model, hidden_size, num_classes=6).to(device)
        
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001) # correct_bias=False # Adam
    loss_fn = nn.CrossEntropyLoss()
    
    classifier, optimizer, train_loader, valid_loader = accelerator.prepare(
        classifier, optimizer, train_loader, valid_loader
    )
    
    # num_training_steps=num_epochs* len(train_loader)
    # progress_bar=tqdm(range(num_training_steps))
    
    for e in range(epoch_start+1, num_epochs+epoch_start+1):
        train_acc, train_loss = 0.0, 0.0
        val_acc, val_loss = 0.0, 0.0
        classifier.train()
        # train_sampler.set_epoch(e)
        
        for batch_id, batch in enumerate(tqdm.notebook.tqdm(train_loader)):

            img = batch[0].to(device)
            label = batch[1].to(device) # .squeeze(1) .float()

            # with torch.cuda.amp.autocast():
            out = classifier(img).squeeze(1)
            loss = loss_fn(out, label)

            train_loss += loss.item()

            accelerator.backward(loss) # loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()
#             optimizer.zero_grad()

            train_acc += calc_accuracy(out, label)

        tot_train_acc = train_acc / (batch_id+1)
        mean_train_loss = train_loss / (batch_id+1)
        train_loss_epoch.append(mean_train_loss)
        train_acc_epoch.append(tot_train_acc)
        accelerator.print("[GPU {}] epoch {} train acc {} loss {}".format(accelerator.local_process_index, e, tot_train_acc, mean_train_loss))

        classifier.eval()
        for batch_id, batch in enumerate(tqdm.notebook.tqdm(valid_loader)):

            img = batch[0].to(device)
            label = batch[1].to(device) # .squeeze(1)
            
            # with torch.cuda.amp.autocast():
            out = classifier(img).squeeze(1)
            loss = loss_fn(out, label)
                
            val_loss += loss.item()
            val_acc += calc_accuracy(out, label)

        tot_acc = val_acc / (batch_id+1)
        mean_val_loss = val_loss / (batch_id+1)
        val_loss_epoch.append(mean_val_loss)
        val_acc_epoch.append(tot_acc)
        accelerator.print("[GPU {}] epoch {} valid acc {} loss {}".format(accelerator.local_process_index, e, tot_acc, mean_val_loss))
        if accelerator.local_process_index==0 and best_val_loss > mean_val_loss:
          clear_output(wait=True)
          print("epoch {} train acc {} validation acc {}".format(e, tot_train_acc, tot_acc))
          loss_epoch_curve(train_loss_epoch, val_loss_epoch, train_acc_epoch, val_acc_epoch)
          best_val_loss = mean_val_loss
          save_checkpoint(e, labels, classifier, optimizer, 
                          train_loss_epoch, val_loss_epoch, train_acc_epoch, val_acc_epoch,
                          model_path, model_name)

In [ ]:
set_seed(42)

In [ ]:
torch.cuda.is_initialized()

In [ ]:
args = (model, hidden_size, labels, model_path, model_name, 10, 'fp16', 32)
notebook_launcher(main_worker, args, num_processes=2)